# Convolutional Time Series
here, we can use 1D convolutions on the time series

In [3]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [5]:
def window_dataset(series, window_size: int, batch_size: int = 32):
    """
    From a time series, create a set of learnable windows.

    Parameters
    ---------
    Series
        The series to learn
    Window size
        the size of window to learn from
    batch size
        The size of each batch in the dataset
    """
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # We want windows of the same length!
    # We also want batches of the window size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # Use the previous window_size-1 data points to predict the next one, and shuffle the windows.
    dataset = dataset.map(lambda window: (window[:-1], window[-1:]))
    dataset = dataset.shuffle(len(series))
    return dataset.batch(batch_size).prefetch(1)


def model_forecast(model, series, window_size: int):
    """
    Use a model to predict a small range into the future.
    """
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    return model.predict(ds)


def seq2seq_window_dataset(
    series, window_size: int, batch_size: int = 32, shuffle_buffer: int = 1000
):
    """
    From a time series, create a set of learnable windows.

    Parameters
    ---------
    Series
        The series to learn
    Window size
        the size of window to learn from
    batch size
        The size of each batch in the dataset
    shuffle_buffer
        Don't eat all of my RAM while shuffling
    """
    series = tf.expand_dims(series, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # We want windows of the same length!
    # We also want batches of the window size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # Use the previous window_size-1 data points to predict the next one, and shuffle the windows.
    dataset = dataset.map(lambda window: (window[:-1], window[1:]))
    dataset = dataset.shuffle(shuffle_buffer)
    return dataset.batch(batch_size).prefetch(1)


def sequential_window_dataset(series, window_size):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=window_size, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(1).prefetch(1)

<IPython.core.display.Javascript object>

In [6]:
def trend(time: np.array, slope: float) -> np.array:
    return slope * time


def seasonal_pattern(season_time: np.array):
    """
    This is arbitrary -- there could be multiple seasonal patterns.
    """
    return np.where(
        season_time < 0.4, np.cos(season_time * 2 * np.pi), np.exp(-3 * season_time)
    )


def seasonality(time: np.array, period: float, amplitude: float = 1, phase: float = 0):
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def white_noise(time: np.array, noise_level: float, seed: int = 31415926):
    rng = np.random.default_rng(seed=seed)
    return rng.random(time.shape[0]) * noise_level

<IPython.core.display.Javascript object>

In [21]:
time = np.arange(4 * 365 + 1)
series = (
    seasonality(time, period=365, amplitude=40)
    + white_noise(time, 10)
    + trend(time, 0.1)
)

split_time = 1000

time_train = time[:split_time]
x_train = series[:split_time]

time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 30
batch_size = 128
train_set = seq2seq_window_dataset(x_train, window_size, batch_size=128)
valid_set = seq2seq_window_dataset(x_valid, window_size, batch_size=128)

<IPython.core.display.Javascript object>

In [22]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

<IPython.core.display.Javascript object>

In [23]:
conv_model = sful_model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv1D(
            filters=32,
            kernel_size=5,
            strides=1,
            padding="causal",
            activation="relu",
            input_shape=[None, 1],
        ),
        tf.keras.layers.LSTM(
            100,
            return_sequences=True,
        ),
        tf.keras.layers.LSTM(100, return_sequences=True),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 200.0),
    ]
)

optimizer = tf.keras.optimizers.SGD(lr=1e-7, momentum=0.9)
conv_model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])

<IPython.core.display.Javascript object>

In [24]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10)
reset_states = ResetStatesCallback()

conv_model.fit(
    train_set,
    epochs=500,
    validation_data=valid_set,
    callbacks=[early_stopping, reset_states],
)

Epoch 1/500
8/8 [==============================] - 8s 154ms/step - loss: 64.6123 - mae: 65.1108 - val_loss: 145.1942 - val_mae: 145.6942
Epoch 2/500
8/8 [==============================] - 0s 26ms/step - loss: 61.3282 - mae: 61.8269 - val_loss: 137.9950 - val_mae: 138.4950
Epoch 3/500
8/8 [==============================] - 0s 26ms/step - loss: 57.8101 - mae: 58.3089 - val_loss: 129.8761 - val_mae: 130.3761
Epoch 4/500
8/8 [==============================] - 0s 26ms/step - loss: 54.4510 - mae: 54.9499 - val_loss: 121.8116 - val_mae: 122.3116
Epoch 5/500
8/8 [==============================] - 0s 26ms/step - loss: 48.7759 - mae: 49.2745 - val_loss: 114.1396 - val_mae: 114.6396
Epoch 6/500
8/8 [==============================] - 0s 29ms/step - loss: 46.1754 - mae: 46.6737 - val_loss: 106.8757 - val_mae: 107.3757
Epoch 7/500
8/8 [==============================] - 0s 27ms/step - loss: 43.5026 - mae: 44.0001 - val_loss: 100.7553 - val_mae: 101.2553
Epoch 8/500
8/8 [==============================

8/8 [==============================] - 0s 28ms/step - loss: 10.5649 - mae: 11.0481 - val_loss: 34.1248 - val_mae: 34.6219
Epoch 62/500
8/8 [==============================] - 0s 29ms/step - loss: 10.5543 - mae: 11.0379 - val_loss: 33.7553 - val_mae: 34.2523
Epoch 63/500
8/8 [==============================] - 0s 29ms/step - loss: 10.0060 - mae: 10.4884 - val_loss: 33.3199 - val_mae: 33.8168
Epoch 64/500
8/8 [==============================] - 0s 26ms/step - loss: 10.1260 - mae: 10.6092 - val_loss: 33.0503 - val_mae: 33.5473
Epoch 65/500
8/8 [==============================] - 0s 29ms/step - loss: 9.9299 - mae: 10.4130 - val_loss: 32.7179 - val_mae: 33.2149
Epoch 66/500
8/8 [==============================] - 0s 28ms/step - loss: 9.7704 - mae: 10.2537 - val_loss: 32.5012 - val_mae: 32.9987
Epoch 67/500
8/8 [==============================] - 0s 27ms/step - loss: 9.6708 - mae: 10.1537 - val_loss: 32.2611 - val_mae: 32.7588
Epoch 68/500
8/8 [==============================] - 0s 29ms/step - loss

8/8 [==============================] - 0s 28ms/step - loss: 6.6031 - mae: 7.0797 - val_loss: 24.6265 - val_mae: 25.1212
Epoch 123/500
8/8 [==============================] - 0s 27ms/step - loss: 6.3237 - mae: 6.7999 - val_loss: 24.4510 - val_mae: 24.9455
Epoch 124/500
8/8 [==============================] - 0s 27ms/step - loss: 6.4497 - mae: 6.9270 - val_loss: 24.4566 - val_mae: 24.9513
Epoch 125/500
8/8 [==============================] - 0s 28ms/step - loss: 6.2758 - mae: 6.7516 - val_loss: 24.3539 - val_mae: 24.8485
Epoch 126/500
8/8 [==============================] - 0s 28ms/step - loss: 6.3200 - mae: 6.7966 - val_loss: 24.4028 - val_mae: 24.8976
Epoch 127/500
8/8 [==============================] - 0s 27ms/step - loss: 6.2211 - mae: 6.6972 - val_loss: 24.2614 - val_mae: 24.7560
Epoch 128/500
8/8 [==============================] - 0s 26ms/step - loss: 6.1819 - mae: 6.6569 - val_loss: 24.2751 - val_mae: 24.7697
Epoch 129/500
8/8 [==============================] - 0s 28ms/step - loss: 6.

8/8 [==============================] - 0s 26ms/step - loss: 5.3998 - mae: 5.8735 - val_loss: 21.1432 - val_mae: 21.6377
Epoch 184/500
8/8 [==============================] - 0s 28ms/step - loss: 5.4001 - mae: 5.8732 - val_loss: 20.8444 - val_mae: 21.3388
Epoch 185/500
8/8 [==============================] - 0s 27ms/step - loss: 5.3606 - mae: 5.8347 - val_loss: 20.9048 - val_mae: 21.3994
Epoch 186/500
8/8 [==============================] - 0s 26ms/step - loss: 5.4071 - mae: 5.8807 - val_loss: 20.9531 - val_mae: 21.4476
Epoch 187/500
8/8 [==============================] - 0s 27ms/step - loss: 5.2641 - mae: 5.7377 - val_loss: 20.7313 - val_mae: 21.2257
Epoch 188/500
8/8 [==============================] - 0s 27ms/step - loss: 5.4201 - mae: 5.8941 - val_loss: 20.9346 - val_mae: 21.4291
Epoch 189/500
8/8 [==============================] - 0s 28ms/step - loss: 5.4012 - mae: 5.8746 - val_loss: 20.6258 - val_mae: 21.1202
Epoch 190/500
8/8 [==============================] - 0s 27ms/step - loss: 5.

8/8 [==============================] - 0s 28ms/step - loss: 4.9422 - mae: 5.4149 - val_loss: 18.9499 - val_mae: 19.4435
Epoch 245/500
8/8 [==============================] - 0s 28ms/step - loss: 4.9398 - mae: 5.4131 - val_loss: 19.0021 - val_mae: 19.4957
Epoch 246/500
8/8 [==============================] - 0s 28ms/step - loss: 4.9125 - mae: 5.3859 - val_loss: 18.9543 - val_mae: 19.4479
Epoch 247/500
8/8 [==============================] - 0s 26ms/step - loss: 4.9424 - mae: 5.4151 - val_loss: 18.9290 - val_mae: 19.4226
Epoch 248/500
8/8 [==============================] - 0s 28ms/step - loss: 4.8761 - mae: 5.3488 - val_loss: 18.8843 - val_mae: 19.3779
Epoch 249/500
8/8 [==============================] - 0s 26ms/step - loss: 4.8691 - mae: 5.3423 - val_loss: 18.7488 - val_mae: 19.2420
Epoch 250/500
8/8 [==============================] - 0s 27ms/step - loss: 4.8127 - mae: 5.2852 - val_loss: 18.9180 - val_mae: 19.4116
Epoch 251/500
8/8 [==============================] - 0s 27ms/step - loss: 4.

8/8 [==============================] - 0s 27ms/step - loss: 4.7144 - mae: 5.1862 - val_loss: 17.6962 - val_mae: 18.1892
Epoch 306/500
8/8 [==============================] - 0s 28ms/step - loss: 4.5938 - mae: 5.0653 - val_loss: 17.7981 - val_mae: 18.2912
Epoch 307/500
8/8 [==============================] - 0s 27ms/step - loss: 4.6251 - mae: 5.0955 - val_loss: 17.6790 - val_mae: 18.1721
Epoch 308/500
8/8 [==============================] - 0s 29ms/step - loss: 4.6946 - mae: 5.1659 - val_loss: 17.6852 - val_mae: 18.1784
Epoch 309/500
8/8 [==============================] - 0s 27ms/step - loss: 4.8340 - mae: 5.3069 - val_loss: 17.8422 - val_mae: 18.3351
Epoch 310/500
8/8 [==============================] - 0s 27ms/step - loss: 4.6008 - mae: 5.0712 - val_loss: 17.4660 - val_mae: 17.9587
Epoch 311/500
8/8 [==============================] - 0s 27ms/step - loss: 4.5993 - mae: 5.0705 - val_loss: 17.7679 - val_mae: 18.2609
Epoch 312/500
8/8 [==============================] - 0s 27ms/step - loss: 4.

8/8 [==============================] - 0s 27ms/step - loss: 4.4383 - mae: 4.9088 - val_loss: 16.8460 - val_mae: 17.3384
Epoch 367/500
8/8 [==============================] - 0s 28ms/step - loss: 4.4831 - mae: 4.9541 - val_loss: 16.9439 - val_mae: 17.4362
Epoch 368/500
8/8 [==============================] - 0s 26ms/step - loss: 4.3799 - mae: 4.8505 - val_loss: 16.8041 - val_mae: 17.2965
Epoch 369/500
8/8 [==============================] - 0s 26ms/step - loss: 4.4604 - mae: 4.9305 - val_loss: 16.8851 - val_mae: 17.3775
Epoch 370/500
8/8 [==============================] - 0s 28ms/step - loss: 4.4555 - mae: 4.9256 - val_loss: 16.8323 - val_mae: 17.3247
Epoch 371/500
8/8 [==============================] - 0s 28ms/step - loss: 4.4393 - mae: 4.9097 - val_loss: 16.7334 - val_mae: 17.2258
Epoch 372/500
8/8 [==============================] - 0s 27ms/step - loss: 4.3705 - mae: 4.8422 - val_loss: 16.9126 - val_mae: 17.4048
Epoch 373/500
8/8 [==============================] - 0s 27ms/step - loss: 4.

8/8 [==============================] - 0s 27ms/step - loss: 4.2914 - mae: 4.7609 - val_loss: 16.2079 - val_mae: 16.7001
Epoch 428/500
8/8 [==============================] - 0s 29ms/step - loss: 4.3123 - mae: 4.7822 - val_loss: 16.2238 - val_mae: 16.7160
Epoch 429/500
8/8 [==============================] - 0s 28ms/step - loss: 4.2730 - mae: 4.7422 - val_loss: 16.0925 - val_mae: 16.5845
Epoch 430/500
8/8 [==============================] - 0s 28ms/step - loss: 4.2983 - mae: 4.7682 - val_loss: 16.1715 - val_mae: 16.6636
Epoch 431/500
8/8 [==============================] - 0s 29ms/step - loss: 4.3190 - mae: 4.7893 - val_loss: 16.1728 - val_mae: 16.6650
Epoch 432/500
8/8 [==============================] - 0s 27ms/step - loss: 4.3352 - mae: 4.8049 - val_loss: 16.0655 - val_mae: 16.5575
Epoch 433/500
8/8 [==============================] - 0s 26ms/step - loss: 4.3035 - mae: 4.7732 - val_loss: 16.1982 - val_mae: 16.6904
Epoch 434/500
8/8 [==============================] - 0s 28ms/step - loss: 4.

<IPython.core.display.Javascript object>

In [ ]:
wave_model = tf.keras.models.Sequential()
wave_model.add(tf.keras.layers.InputLayer(input_shape=[None, 1]))
for dilation_rate in (1, 2, 4, 8, 16, 32):
    wave_model.add(
        tf.keras.layers.Conv1D(
            filters=32,
            kernel_size=2,
            strides=1,
            dilation_rate=dilation_rate,
            padding="causal",
            activation="relu",
        )
    )
wave_model.add(tf.keras.layers.Conv1D(filters=1, kernel_size=1))

wave_model.compile(loss=tf.keras.losses.Huber(),
                  optimizer=tf.keras.optimizers.Adam(lr=3e-4),
                  metrics=["mae"])
history = wave_model.fit(train_set, epochs=500,
                        validation_data=valid_set,
                        callbacks=[early_stopping])

Epoch 1/500
8/8 [==============================] - 2s 111ms/step - loss: 61.3111 - mae: 61.8102 - val_loss: 130.9514 - val_mae: 131.4514
Epoch 2/500
8/8 [==============================] - 0s 20ms/step - loss: 60.0683 - mae: 60.5674 - val_loss: 127.1801 - val_mae: 127.6801
Epoch 3/500
8/8 [==============================] - 0s 20ms/step - loss: 57.7945 - mae: 58.2937 - val_loss: 122.0164 - val_mae: 122.5164
Epoch 4/500
8/8 [==============================] - 0s 19ms/step - loss: 55.7058 - mae: 56.2047 - val_loss: 114.3263 - val_mae: 114.8264
Epoch 5/500
8/8 [==============================] - 0s 19ms/step - loss: 51.3533 - mae: 51.8523 - val_loss: 101.8287 - val_mae: 102.3287
Epoch 6/500
8/8 [==============================] - 0s 21ms/step - loss: 45.3070 - mae: 45.8061 - val_loss: 82.1769 - val_mae: 82.6769
Epoch 7/500
8/8 [==============================] - 0s 20ms/step - loss: 34.2866 - mae: 34.7849 - val_loss: 53.2097 - val_mae: 53.7066
Epoch 8/500
8/8 [==============================] - 

In [ ]:
cnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)